## Required imports

In [1]:
import unittest
from math import inf

from cobra import io
from cobra import Model
from cobra import Reaction
from reframed import load_cbmodel, Community, Environment, FBA, SteadyCom, save_cbmodel, CBReaction, ReactionType
from tests.test_reframed import MIN_GROWTH, pFBA, FVA
from reframed.community.simulation import simulate

import pandas as pd


### Import organism models

In [2]:

acaldus_model = load_cbmodel("./Acidithiobacillus_caldus_SM-1/Curated/Acaldus.xml")
acaldus_model.id = "acaldus"

aferrooxidans_model = load_cbmodel('./Acidimicrobium_ferrooxidans_DSM_10331/Curated/Aferrooxidans.xml')
aferrooxidans_model.id = "aferrooxidans"


### Open all drains from the organism models

In [3]:

exchange_reactions_acu = acaldus_model.get_exchange_reactions()


for rxn in exchange_reactions_acu:
    exchange_reaction_object_acu = acaldus_model.reactions.get(rxn)
    exchange_reaction_object_acu.lb = -inf

    
exchange_reactions_afe = aferrooxidans_model.get_exchange_reactions()

for rxn in exchange_reactions_afe:
    exchange_reaction_object_afe = aferrooxidans_model.reactions.get(rxn)
    exchange_reaction_object_afe.lb = -inf
    


### Create community model and save it

In [4]:

basePath="./comModel.xml"

community = Community('comModel', [acaldus_model, aferrooxidans_model])

community_model = community.merged_model

save_cbmodel(community_model, basePath, flavor='fbc2')

### Implement community medium

In [22]:
'''
constraint_model = load_cbmodel('Acaldus_exchange_reactions_constraints.xml')
medium = Environment.from_model(constraint_model)

medium.apply(community_model)
'''

    
minimal_medium = ["EX_C00014__extr","EX_C00007__extr","EX_C00009__extr","EX_C00087__extr","EX_C14818__extr","EX_C00001__extr",
                    "EX_C00059__extr"]
carbon_source = ["EX_C00011__extr"]

constraints = {}

medium = []

for exchange_reaction in community_model.get_exchange_reactions():
    exchange_reaction_object = community_model.reactions.get(exchange_reaction)
    exchange_reaction_object.lb = 0
    for reaction in minimal_medium:
        if reaction in exchange_reaction_object.id:
            exchange_reaction_object.lb = -inf
    for reaction in carbon_source:
        if reaction in exchange_reaction_object.id:
            exchange_reaction_object.lb = -0.7106
    if exchange_reaction_object.lb != 0:
        constraints[exchange_reaction_object.id] = (exchange_reaction_object.lb,exchange_reaction_object.ub)
        for i in community_model.metabolites.items():
            if i[0] == str(exchange_reaction_object.get_substrates())[2:-2]:
                met_name = str(i[1])
#                 met_name = met_name.split('_')[1]
        comp = (exchange_reaction_object.id, str(exchange_reaction_object.get_substrates())[2:-2], met_name, exchange_reaction_object.lb)
        medium.append(comp)
        
medium_df = pd.DataFrame(medium,columns=["Reaction ID","Metabolite ID", "Metabolite Name", "Flux"])
# medium_df.to_csv('community_medium.csv')

print(medium_df)
print('')
print(constraints)


         Reaction ID   Metabolite ID Metabolite Name    Flux
0  R_EX_C00059__extr  M_C00059__extr         Sulfate    -inf
1  R_EX_C00009__extr  M_C00009__extr  Orthophosphate    -inf
2  R_EX_C00011__extr  M_C00011__extr             CO2 -0.7106
3  R_EX_C00014__extr  M_C00014__extr         Ammonia    -inf
4  R_EX_C00087__extr  M_C00087__extr          Sulfur    -inf
5  R_EX_C14818__extr  M_C14818__extr            Fe2+    -inf
6  R_EX_C00001__extr  M_C00001__extr             H2O    -inf
7  R_EX_C00007__extr  M_C00007__extr          Oxygen    -inf

{'R_EX_C00059__extr': (-inf, inf), 'R_EX_C00009__extr': (-inf, inf), 'R_EX_C00011__extr': (-0.7106, inf), 'R_EX_C00014__extr': (-inf, inf), 'R_EX_C00087__extr': (-inf, inf), 'R_EX_C14818__extr': (-inf, inf), 'R_EX_C00001__extr': (-inf, inf), 'R_EX_C00007__extr': (-inf, inf)}


### Simulate the community model and check organisms and community's biomass fva values as well as  community's exchanged metabolites

In [13]:
# from .variability import FVA

'''
constraint_model = load_cbmodel('Acaldus_exchange_reactions_constraints.xml')
medium = Environment.from_model(constraint_model)

medium.apply(community_model)
'''

print(community_model.get_objective())

sol = pFBA(community_model, constraints=constraints)

sol1 = FVA(community_model, constraints=constraints)


print(sol1['community_growth'])

print(sol1['R_e_Biomass__cytop_acaldus'])
print(sol1['R_e_Biomass__cytop_aferrooxidans'])

print('')


#print(sol)

#print(community.merged_model.biomass_reaction)

#res = simulate(community, objective='community_growth', constraints=constraints)
#print(res.values)

com_exp = []
com_imp = []

for exchange_reaction in community_model.get_exchange_reactions():
    exchange_reaction_object = community_model.reactions.get(exchange_reaction)
    if sol.values[str(exchange_reaction_object.id)] > 0 :
        for i in community_model.metabolites.items():
            if i[0] == str(exchange_reaction_object.get_substrates())[2:-2]:
                met_name = str(i[1])
#                 met_name = met_name.split('_')[1]   
        line = (exchange_reaction_object.id, str(exchange_reaction_object.get_substrates())[2:-2], met_name, sol.values[str(exchange_reaction_object.id)])
        com_exp.append(line)
print('')
for exchange_reaction in community_model.get_exchange_reactions():
    exchange_reaction_object = community_model.reactions.get(exchange_reaction)
    if sol.values[str(exchange_reaction_object.id)] < 0 :
        for i in community_model.metabolites.items():
            if i[0] == str(exchange_reaction_object.get_substrates())[2:-2]:
                met_name = str(i[1])
#                 met_name = met_name.split('_')[1]   
        line = (exchange_reaction_object.id, str(exchange_reaction_object.get_substrates())[2:-2], met_name, sol.values[str(exchange_reaction_object.id)])
        com_imp.append(line)

com_exp_df = pd.DataFrame(com_exp,columns=["Reaction ID","Metabolite ID", "Metabolite Name", "Flux"])
com_imp_df = pd.DataFrame(com_imp,columns=["Reaction ID","Metabolite ID", "Metabolite Name", "Flux"])



# com_exp_df.to_csv('community_export.csv')
# com_imp_df.to_csv('community_import.csv')        

print(com_exp_df)
print('')
print(com_imp_df)


{'community_growth': 1}
[0.0, 0.0206260248549022]
[0.0, 0.018517645789206102]
[0.0, 0.02062602485490227]


            Reaction ID      Metabolite ID    Metabolite Name       Flux
0     R_EX_C05198__extr     M_C05198__extr  5'-Deoxyadenosine   0.000147
1     R_EX_C00059__extr     M_C00059__extr            Sulfate  21.192711
2     R_EX_C00237__extr     M_C00237__extr                 CO   0.000147
3     R_EX_C00080__extr     M_C00080__extr                 H+   9.058429
4  R_EX_e_Biomass__extr  M_e_Biomass__extr          e-Biomass   0.020626

         Reaction ID   Metabolite ID Metabolite Name       Flux
0  R_EX_C00009__extr  M_C00009__extr  Orthophosphate  -0.011927
1  R_EX_C00011__extr  M_C00011__extr             CO2  -0.710600
2  R_EX_C00014__extr  M_C00014__extr         Ammonia  -0.179777
3  R_EX_C00087__extr  M_C00087__extr          Sulfur -21.195519
4  R_EX_C14818__extr  M_C14818__extr            Fe2+  -0.000147
5  R_EX_C00001__extr  M_C00001__extr             H2O -25.943312
6  R_E

### Check exchanged metabolites within the community

In [14]:


exchange_acaldus_imp=[]
exchange_acaldus_exp=[]

exchange_aferrooxidans_imp=[]
exchange_aferrooxidans_exp=[]


for exchange_reaction in community_model.get_exchange_reactions():
    exchange_reaction_object = community_model.reactions.get(exchange_reaction)
    reaction_metabolites = exchange_reaction_object.stoichiometry.keys()
    for metabolite in reaction_metabolites:
        if metabolite:
            reactions = community_model.get_metabolite_reactions(metabolite)
            reac_cons = community_model.get_metabolite_consumers(metabolite)
            reac_prod = community_model.get_metabolite_producers(metabolite)
            for reaction in reactions:
                if "_acaldus" in reaction and sol.values[str(reaction)] != 0:
                    if reaction in reac_prod and sol.values[str(reaction)] > 0:
                        reac_ap = str(community_model.reactions.get(reaction))
                        for i in community_model.metabolites.items():
                            if i[0] == metabolite:
                                met_name =str(i[1])
#                                 met_name = met_name.split('_')[1]
                        flux_ap = (reac_ap.split(":")[0], metabolite, met_name, sol.values[str(reaction)])
                        exchange_acaldus_exp.append(flux_ap)
                    if reaction in reac_prod and sol.values[str(reaction)] < 0:
                        reac_ac = str(community_model.reactions.get(reaction))
                        for i in community_model.metabolites.items():
                            if i[0] == metabolite:
                                met_name = str(i[1])
#                                 met_name = met_name.split('_')[1]
                        flux_ac = (reac_ac.split(":")[0], metabolite,  met_name, sol.values[str(reaction)])
                        exchange_acaldus_imp.append(flux_ac)                    
                    if reaction in reac_cons and sol.values[str(reaction)] < 0:
                        reac_ap = str(community_model.reactions.get(reaction))
                        for i in community_model.metabolites.items():
                            if i[0] == metabolite:
                                met_name = str(i[1])
#                                 met_name = met_name.split('_')[1]
                        flux_ap = (reac_ap.split(":")[0], metabolite, met_name, sol.values[str(reaction)])
                        exchange_acaldus_exp.append(flux_ap)
                    if reaction in reac_cons and sol.values[str(reaction)] > 0:
                        reac_ac = str(community_model.reactions.get(reaction))
                        for i in community_model.metabolites.items():
                            if i[0] == metabolite:
                                met_name = str(i[1])
#                                 met_name = met_name.split('_')[1]
                        flux_ac = (reac_ac.split(":")[0], metabolite, met_name, sol.values[str(reaction)])
                        exchange_acaldus_imp.append(flux_ac)
                if "_aferrooxidans" in reaction and sol.values[str(reaction)] != 0:
                    if reaction in reac_prod and sol.values[str(reaction)] > 0:
                        reac_afp = str(community_model.reactions.get(reaction))
                        for i in community_model.metabolites.items():
                            if i[0] == metabolite:
                                met_name = str(i[1])
#                                 met_name = met_name.split('_')[1]
                        flux_afp = (reac_afp.split(":")[0], metabolite, met_name, sol.values[str(reaction)])
                        exchange_aferrooxidans_exp.append(flux_afp)
                    if reaction in reac_prod and sol.values[str(reaction)] < 0: 
                        reac_afc = str(community_model.reactions.get(reaction))
                        for i in community_model.metabolites.items():
                            if i[0] == metabolite:
                                met_name = str(i[1])
#                                 met_name = met_name.split('_')[1]
                        flux_afc = (reac_afc.split(":")[0], metabolite, met_name, sol.values[str(reaction)])
                        exchange_aferrooxidans_imp.append(flux_afc)                    
                    if reaction in reac_cons and sol.values[str(reaction)] < 0:
                        reac_afp = str(community_model.reactions.get(reaction))
                        for i in community_model.metabolites.items():
                            if i[0] == metabolite:
                                met_name = str(i[1])
#                                 met_name = met_name.split('_')[1]
                        flux_afp = (reac_afp.split(":")[0], metabolite, met_name, sol.values[str(reaction)])
                        exchange_aferrooxidans_exp.append(flux_afp)
                    if reaction in reac_cons and sol.values[str(reaction)] > 0: 
                        reac_afc = str(community_model.reactions.get(reaction))
                        for i in community_model.metabolites.items():
                            if i[0] == metabolite:
                                met_name = str(i[1])
#                                 met_name = met_name.split('_')[1]
                        flux_afc = (reac_afc.split(":")[0], metabolite, met_name, sol.values[str(reaction)])
                        exchange_aferrooxidans_imp.append(flux_afc)
                    

pd.set_option('display.max_colwidth', None)
exchange_acaldus_imp_df = pd.DataFrame(exchange_acaldus_imp,columns=["Reaction ID","Metabolite ID", "Metabolite Name", "Flux"])
exchange_acaldus_exp_df = pd.DataFrame(exchange_acaldus_exp,columns=["Reaction ID","Metabolite ID", "Metabolite Name", "Flux"])
exchange_aferrooxidans_exp_df = pd.DataFrame(exchange_aferrooxidans_exp,columns=["Reaction ID","Metabolite ID", "Metabolite Name", "Flux"])
exchange_aferrooxidans_imp_df = pd.DataFrame(exchange_aferrooxidans_imp,columns=["Reaction ID","Metabolite ID", "Metabolite Name", "Flux"])


af_ac = pd.DataFrame(columns=["Reaction ID","Metabolite ID", "Metabolite Name", "Flux"])

for index, line in exchange_acaldus_imp_df.iterrows():
    if line['Metabolite ID'] in list(exchange_aferrooxidans_exp_df['Metabolite ID']):
        dic = {"Reaction ID": line["Reaction ID"], "Metabolite ID": line["Metabolite ID"], 
               "Metabolite Name": line["Metabolite Name"], "Flux": line["Flux"]}
        af_ac = af_ac.append(dic, ignore_index=True)


ac_af = pd.DataFrame(columns=["Reaction ID","Metabolite ID", "Metabolite Name", "Flux"])
    
for index, line in exchange_aferrooxidans_imp_df.iterrows():
    if line['Metabolite ID'] in list(exchange_acaldus_exp_df['Metabolite ID']):
        dic = {"Reaction ID": line["Reaction ID"], "Metabolite ID": line["Metabolite ID"], 
               "Metabolite Name": line["Metabolite Name"], "Flux": line["Flux"]}
        ac_af = ac_af.append(dic, ignore_index=True)

        
        

# exchange_acaldus_imp_df.to_csv('exchange_acaldus_imp.csv')
# exchange_acaldus_exp_df.to_csv('exchange_acaldus_exp.csv')
# exchange_aferrooxidans_exp_df.to_csv('exchange_aferrooxidans_exp.csv')
# exchange_aferrooxidans_imp_df.to_csv('exchange_aferrooxidans_imp.csv')

# ac_af.to_csv('ac_af.csv')
# af_ac.to_csv('af_ac.csv')



# exchange_acaldus_imp_df
# print('')
# exchange_acaldus_exp_df
# print('')
# exchange_aferrooxidans_exp_df
# print('')
# exchange_aferrooxidans_imp_df

af_ac


,Reaction ID,Metabolite ID,Metabolite Name,Flux
0,R_TR0000239_CYTMEM_outer__outme_acaldus,M_C00283__extr,Hydrogen sulfide,0.071021
1,R_TR0000242_CYTMEM_outer__outme_acaldus,M_C00288__extr,HCO3-,0.202874
2,R_TR0000012_CYTMEM_outer__outme_acaldus,M_C00013__extr,Diphosphate,-0.000251


In [15]:
ac_af

,Reaction ID,Metabolite ID,Metabolite Name,Flux
0,R_TO3001131_CYTMEM__cytmem_aferrooxidans,M_C00350__extr,Phosphatidylethanolamine,0.000509
1,R_TZ5900021_CYTMEM__cytmem_aferrooxidans,M_C00249__extr,Hexadecanoic acid,0.008083
2,R_TO3000701_CYTMEM__cytmem_aferrooxidans,M_C00993__extr,D-Alanyl-D-alanine,0.000772
3,R_TO3001142_CYTMEM__cytmem_aferrooxidans,M_C02737__extr,Phosphatidylserine,0.000048
4,R_TO3001770_CYTMEM__cytmem_aferrooxidans,M_C00669__extr,gamma-L-Glutamyl-L-cysteine,0.073829
5,R_TO3001129_CYTMEM__cytmem_aferrooxidans,M_C00344__extr,Phosphatidylglycerol,0.002000
6,R_TO3001126_CYTMEM__cytmem_aferrooxidans,M_C00416__extr,Phosphatidate,0.000012


### Run SteadyCom 

In [24]:
SteadyCom(community, constraints=constraints)

Community growth: 0.0205078125
acaldus	0.0001659041933591074
aferrooxidans	0.999834095806641

### Convert ID's from kegg to Bigg in order to run smetana

In [7]:
#convert ID's from kegg to Bigg in order to run smetana

import os

import pandas as pd


basePath = "./"

list_models = []

for file in os.listdir(basePath):
    if file.endswith('.csv'):
        id_df = pd.read_csv(os.path.join(basePath, file), sep = ';')
    if file.endswith("s.xml"):
        SBML_FILE = (os.path.join(basePath, file))
        list_models.append(SBML_FILE)

lst = id_df.values.tolist()
dct = dict((x,y) for x, y in lst)

for l in list_models:
    new = open(str(l)[:-4] + "_smetana.xml", 'w')
    with open(l, 'r') as m:
        data = m.read()
        m.close()
    for key, value in dct.items():
        data = data.replace(str(value), str(key))
    new.write(data)
    new.close()




### Run SMETANA

In [9]:
#run smetana

from smetana.interface import main
import pandas as pd
import sys


acaldus = "Acaldus_smetana.xml"
aferrooxidans = "Aferrooxidans_smetana.xml"



main([acaldus,aferrooxidans], mode="detailed", output="./smetana")



### Rearrange the output of smetana


In [31]:

import re

basePath = "./"
file = "id_conversion.csv"

id_df = pd.read_csv(os.path.join(basePath, file), sep = ';')


lst = id_df.values.tolist()
dct = dict((x,y) for x, y in lst)

df = pd.read_csv("smetana_detailed.tsv",sep="\t")
smetana = pd.DataFrame(df.iloc[:,3:])

kegg=[]
met_name = {}

for index, line in smetana.iterrows():
    x = re.sub("^M_", "", line["compound"])
    x = re.sub("__extr$","",x)
    smetana["compound"][index] = x
    kegg.append(dct[x])
    
smetana.insert(2,"Kegg id",kegg)
smetana = smetana.rename(columns={'compound':'Bigg id'})


for i in community_model.metabolites.items():
    x = re.sub("^M_", "", i[0])
    x = re.sub("__.+$","",x)
    if x in kegg:
        name = str(i[1])
        met_name[x] = name


smetana = smetana.reindex(columns=['donor','Bigg id','Kegg id','Metabolite Name','scs','mus','mps','smetana'])

for index, line in smetana.iterrows():
    smetana["Metabolite Name"][index] = met_name[line["Kegg id"]]
    

af_to_ac = smetana[smetana['donor'].str.contains('Aferrooxidans')].loc[:,['Bigg id','Kegg id','Metabolite Name','smetana']]  
ac_to_af = smetana[smetana['donor'].str.contains('Acaldus')].loc[:,['Bigg id','Kegg id','Metabolite Name','smetana']]   

af_to_ac = af_to_ac.sort_values('smetana', ascending=False)
ac_to_af = ac_to_af.sort_values('smetana', ascending=False)

# af_ac.to_csv('af_to_ac_smetana.csv')
# ac_af.to_csv('ac_to_af_smetana.csv')


af_to_ac


<ipython-input-31-eb037838bbfd>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smetana["compound"][index] = x
<ipython-input-31-eb037838bbfd>:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smetana["Metabolite Name"][index] = met_name[line["Kegg id"]]
C:\Users\Utilizador\Anaconda3\envs\community\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, 

,Bigg id,Kegg id,Metabolite Name,smetana
0,cgly,C01419,Cys-Gly,0.93
10,pi,C00009,Orthophosphate,0.54
9,o2,C00007,Oxygen,0.52
2,fe2,C14818,Fe2+,0.50
11,ppi,C00013,Diphosphate,0.46
4,h2o,C00001,H2O,0.16
6,h,C00080,H+,0.10
7,hco3,C00288,HCO3-,0.07
8,nh4,C00014,Ammonia,0.07
1,co2,C00011,CO2,0.05


In [32]:
ac_to_af

,Bigg id,Kegg id,Metabolite Name,smetana
19,o2,C00007,Oxygen,0.60
24,so4,C00059,Sulfate,0.50
25,tsul,C00320,Thiosulfate,0.43
13,fe2,C14818,Fe2+,0.36
16,h2s,C00283,Hydrogen sulfide,0.07
12,co2,C00011,CO2,0.05
18,nh4,C00014,Ammonia,0.04
15,h2o,C00001,H2O,0.03
21,pi,C00009,Orthophosphate,0.03
14,glucys,C00669,gamma-L-Glutamyl-L-cysteine,0.01


### Raw output of smetana

In [33]:
df

,community,medium,receiver,donor,compound,scs,mus,mps,smetana
0,all,minimal,Acaldus_smetana,Aferrooxidans_smetana,M_cgly__extr,1.0,0.93,1,0.93
1,all,minimal,Acaldus_smetana,Aferrooxidans_smetana,M_co2__extr,1.0,0.05,1,0.05
2,all,minimal,Acaldus_smetana,Aferrooxidans_smetana,M_fe2__extr,1.0,0.50,1,0.50
3,all,minimal,Acaldus_smetana,Aferrooxidans_smetana,M_glucys__extr,1.0,0.02,1,0.02
4,all,minimal,Acaldus_smetana,Aferrooxidans_smetana,M_h2o__extr,1.0,0.16,1,0.16
5,all,minimal,Acaldus_smetana,Aferrooxidans_smetana,M_h2s__extr,1.0,0.05,1,0.05
6,all,minimal,Acaldus_smetana,Aferrooxidans_smetana,M_h__extr,1.0,0.10,1,0.10
7,all,minimal,Acaldus_smetana,Aferrooxidans_smetana,M_hco3__extr,1.0,0.07,1,0.07
8,all,minimal,Acaldus_smetana,Aferrooxidans_smetana,M_nh4__extr,1.0,0.07,1,0.07
9,all,minimal,Acaldus_smetana,Aferrooxidans_smetana,M_o2__extr,1.0,0.52,1,0.52


### Add another objective function and ran the previous analysis

In [34]:
#add an alternative community biomass reaction forcing the flux to go through both organisms' biomass reactions
#the stoichiometry is the result of SteadyCom


from cobra import Reaction

reaction = CBReaction('R_Community_growth_alt',
                      name='Alternative Community growth',
                      reversible=False,
                      stoichiometry={'M_e_Biomass__cytop_acaldus': -0.0506, 'M_e_Biomass__cytop_aferrooxidans': -0.9494},
                      reaction_type=ReactionType.EXCHANGE,
                      lb=0,
                      ub=inf)
 

community_model.add_reaction(reaction)



community_model.reactions.R_Community_growth_alt

R_Community_growth_alt: 0.0506 M_e_Biomass__cytop_acaldus + 0.9494 M_e_Biomass__cytop_aferrooxidans --> 

In [35]:
sol = pFBA(community_model, constraints=constraints, objective='R_Community_growth_alt')

print(sol.values['R_Community_growth_alt'])

print(sol.values['community_growth'])

print(sol.values['R_e_Biomass__cytop_acaldus'])
print(sol.values['R_e_Biomass__cytop_aferrooxidans'])


0.02050787475418875
0.02050787475418875
0.0010376984626022977
0.01947017629158645


In [36]:
com_exp = []
com_imp = []

for exchange_reaction in community_model.get_exchange_reactions():
    exchange_reaction_object = community_model.reactions.get(exchange_reaction)
    if sol.values[str(exchange_reaction_object.id)] > 0 :
        for i in community_model.metabolites.items():
            if i[0] == str(exchange_reaction_object.get_substrates())[2:-2]:
                met_name = str(i[1])
#                 met_name = met_name.split('_')[1]   
        line = (exchange_reaction_object.id, str(exchange_reaction_object.get_substrates())[2:-2], met_name, sol.values[str(exchange_reaction_object.id)])
        com_exp.append(line)

for exchange_reaction in community_model.get_exchange_reactions():
    exchange_reaction_object = community_model.reactions.get(exchange_reaction)
    if sol.values[str(exchange_reaction_object.id)] < 0 :
        for i in community_model.metabolites.items():
            if i[0] == str(exchange_reaction_object.get_substrates())[2:-2]:
                met_name = str(i[1])
#                 met_name = met_name.split('_')[1]   
        line = (exchange_reaction_object.id, str(exchange_reaction_object.get_substrates())[2:-2], met_name, sol.values[str(exchange_reaction_object.id)])
        com_imp.append(line)

com_exp_df = pd.DataFrame(com_exp,columns=["Reaction ID","Metabolite ID", "Metabolite Name", "Flux"])
com_imp_df = pd.DataFrame(com_imp,columns=["Reaction ID","Metabolite ID", "Metabolite Name", "Flux"])

# com_exp_df.to_csv('com_exp_df_forced.csv')
# com_imp_df.to_csv('com_imp_df_forced.csv')



print(com_exp_df)
print('')
print(com_imp_df)

              Reaction ID  \
0       R_EX_C05198__extr   
1       R_EX_C00059__extr   
2       R_EX_C00160__extr   
3       R_EX_C00237__extr   
4       R_EX_C00080__extr   
5  R_Community_growth_alt   

                                                    Metabolite ID  \
0                                                  M_C05198__extr   
1                                                  M_C00059__extr   
2                                                  M_C00160__extr   
3                                                  M_C00237__extr   
4                                                  M_C00080__extr   
5  M_e_Biomass__cytop_acaldus', 'M_e_Biomass__cytop_aferrooxidans   

     Metabolite Name       Flux  
0  5'-Deoxyadenosine   0.000144  
1            Sulfate  21.196760  
2          Glycolate   0.000006  
3                 CO   0.000144  
4                 H+   9.078089  
5                 H+   0.020508  

         Reaction ID   Metabolite ID Metabolite Name       Flux
0  R_EX_C

In [37]:
exchange_acaldus_imp=[]
exchange_acaldus_exp=[]

exchange_aferrooxidans_imp=[]
exchange_aferrooxidans_exp=[]


for exchange_reaction in community_model.get_exchange_reactions():
    exchange_reaction_object = community_model.reactions.get(exchange_reaction)
    reaction_metabolites = exchange_reaction_object.stoichiometry.keys()
    for metabolite in reaction_metabolites:
        if metabolite:
            reactions = community_model.get_metabolite_reactions(metabolite)
            reac_cons = community_model.get_metabolite_consumers(metabolite)
            reac_prod = community_model.get_metabolite_producers(metabolite)
            for reaction in reactions:
                
                if "_acaldus" in reaction and sol.values[str(reaction)] != 0:
                    if reaction in reac_prod and sol.values[str(reaction)] > 0:
                        reac_ap = str(community_model.reactions.get(reaction))
                        for i in community_model.metabolites.items():
                            if i[0] == metabolite:
                                met_name =str(i[1])
#                                 met_name = met_name.split('_')[1]
                        if sol.values[str(reaction)] < 0:
                            reaction_flux = sol.values[str(reaction)]
                            reaction_flux = reaction_flux * -1
                        else:
                            reaction_flux = sol.values[str(reaction)]
                        flux_ap = (metabolite, met_name, reaction_flux)
                        exchange_acaldus_exp.append(flux_ap)
                    
                    if reaction in reac_prod and sol.values[str(reaction)] < 0:
                        reac_ac = str(community_model.reactions.get(reaction))
                        for i in community_model.metabolites.items():
                            if i[0] == metabolite:
                                met_name = str(i[1])
#                                 met_name = met_name.split('_')[1]
                        if sol.values[str(reaction)] < 0:
                            reaction_flux = sol.values[str(reaction)]
                            reaction_flux = reaction_flux * -1
                        else:
                            reaction_flux = sol.values[str(reaction)]
                        flux_ac = (metabolite, met_name, reaction_flux)
                        exchange_acaldus_imp.append(flux_ac)                    
                    
                    if reaction in reac_cons and sol.values[str(reaction)] < 0:
                        reac_ap = str(community_model.reactions.get(reaction))
                        for i in community_model.metabolites.items():
                            if i[0] == metabolite:
                                met_name = str(i[1])
#                                 met_name = met_name.split('_')[1]
                        if sol.values[str(reaction)] < 0:
                            reaction_flux = sol.values[str(reaction)]
                            reaction_flux = reaction_flux * -1
                        else:
                            reaction_flux = sol.values[str(reaction)]
                        flux_ap = (metabolite, met_name, reaction_flux)
                        exchange_acaldus_exp.append(flux_ap)
                    
                    if reaction in reac_cons and sol.values[str(reaction)] > 0:
                        reac_ac = str(community_model.reactions.get(reaction))
                        for i in community_model.metabolites.items():
                            if i[0] == metabolite:
                                met_name = str(i[1])
#                                 met_name = met_name.split('_')[1]
                        if sol.values[str(reaction)] < 0:
                            reaction_flux = sol.values[str(reaction)]
                            reaction_flux = reaction_flux * -1
                        else:
                            reaction_flux = sol.values[str(reaction)]
                        flux_ac = (metabolite, met_name, reaction_flux)
                        exchange_acaldus_imp.append(flux_ac)
                
                
                if "_aferrooxidans" in reaction and sol.values[str(reaction)] != 0:
                    if reaction in reac_prod and sol.values[str(reaction)] > 0:
                        reac_afp = str(community_model.reactions.get(reaction))
                        for i in community_model.metabolites.items():
                            if i[0] == metabolite:
                                met_name = str(i[1])
#                                 met_name = met_name.split('_')[1]
                        if sol.values[str(reaction)] < 0:
                            reaction_flux = sol.values[str(reaction)]
                            reaction_flux = reaction_flux * -1
                        else:
                            reaction_flux = sol.values[str(reaction)]
                        flux_afp = (metabolite, met_name, reaction_flux)
                        exchange_aferrooxidans_exp.append(flux_afp)
                    
                    if reaction in reac_prod and sol.values[str(reaction)] < 0: 
                        reac_afc = str(community_model.reactions.get(reaction))
                        for i in community_model.metabolites.items():
                            if i[0] == metabolite:
                                met_name = str(i[1])
#                                 met_name = met_name.split('_')[1]
                        if sol.values[str(reaction)] < 0:
                            reaction_flux = sol.values[str(reaction)]
                            reaction_flux = reaction_flux * -1
                        else:
                            reaction_flux = sol.values[str(reaction)]
                        flux_afc = (metabolite, met_name, reaction_flux)
                        exchange_aferrooxidans_imp.append(flux_afc)                    
                    
                    if reaction in reac_cons and sol.values[str(reaction)] < 0:
                        reac_afp = str(community_model.reactions.get(reaction))
                        for i in community_model.metabolites.items():
                            if i[0] == metabolite:
                                met_name = str(i[1])
#                                 met_name = met_name.split('_')[1]
                        if sol.values[str(reaction)] < 0:
                            reaction_flux = sol.values[str(reaction)]
                            reaction_flux = reaction_flux * -1
                        else:
                            reaction_flux = sol.values[str(reaction)]
                        flux_afp = (metabolite, met_name, reaction_flux)
                        exchange_aferrooxidans_exp.append(flux_afp)
                    
                    if reaction in reac_cons and sol.values[str(reaction)] > 0: 
                        reac_afc = str(community_model.reactions.get(reaction))
                        for i in community_model.metabolites.items():
                            if i[0] == metabolite:
                                met_name = str(i[1])
#                                 met_name = met_name.split('_')[1]
                        if sol.values[str(reaction)] < 0:
                            reaction_flux = sol.values[str(reaction)]
                            reaction_flux = reaction_flux * -1
                        else:
                            reaction_flux = sol.values[str(reaction)]
                        flux_afc = (metabolite, met_name, reaction_flux)
                        exchange_aferrooxidans_imp.append(flux_afc)
                    

pd.set_option('display.max_colwidth', None)
exchange_acaldus_imp_df = pd.DataFrame(exchange_acaldus_imp,columns=["Metabolite ID", "Metabolite Name", "Flux"])
exchange_acaldus_imp_df = exchange_acaldus_imp_df.sort_values(['Flux'],ascending=False)

exchange_acaldus_exp_df = pd.DataFrame(exchange_acaldus_exp,columns=["Metabolite ID", "Metabolite Name", "Flux"])
exchange_acaldus_exp_df = exchange_acaldus_exp_df.sort_values(['Flux'],ascending=False)

ac_com = pd.concat([exchange_acaldus_imp_df,exchange_acaldus_exp_df])
# ac_com.to_csv('ac_com.csv')

print(pd.concat([exchange_acaldus_imp_df,exchange_acaldus_exp_df]))

exchange_aferrooxidans_exp_df = pd.DataFrame(exchange_aferrooxidans_exp,columns=["Metabolite ID", "Metabolite Name", "Flux"])
exchange_aferrooxidans_exp_df = exchange_aferrooxidans_exp_df.sort_values(['Flux'],ascending=False)


exchange_aferrooxidans_imp_df = pd.DataFrame(exchange_aferrooxidans_imp,columns=["Metabolite ID", "Metabolite Name", "Flux"])
exchange_aferrooxidans_imp_df = exchange_aferrooxidans_imp_df.sort_values(['Flux'],ascending=False)



af_com = pd.concat([exchange_aferrooxidans_imp_df,exchange_aferrooxidans_exp_df])
# af_com.to_csv('af_com.csv')

print('')

print(pd.concat([exchange_aferrooxidans_imp_df,exchange_aferrooxidans_exp_df]))



af_ac = pd.DataFrame(columns=["Metabolite ID", "Metabolite Name", "Flux"])

for index, line in exchange_acaldus_imp_df.iterrows():
    if line['Metabolite ID'] in list(exchange_aferrooxidans_exp_df['Metabolite ID']):
        dic = {"Metabolite ID": line["Metabolite ID"], 
               "Metabolite Name": line["Metabolite Name"], "Flux": line["Flux"]}
        af_ac = af_ac.append(dic, ignore_index=True)


ac_af = pd.DataFrame(columns=["Metabolite ID", "Metabolite Name", "Flux"])
    
for index, line in exchange_aferrooxidans_imp_df.iterrows():
    if line['Metabolite ID'] in list(exchange_acaldus_exp_df['Metabolite ID']):
        dic = {"Metabolite ID": line["Metabolite ID"], 
               "Metabolite Name": line["Metabolite Name"], "Flux": line["Flux"]}
        ac_af = ac_af.append(dic, ignore_index=True)

# af_ac.to_csv('af_ac_forced.csv')
# ac_af.to_csv('ac_af_forced.csv')

print('')        
print(af_ac)
print('')
print(ac_af)       

        


                 Metabolite ID              Metabolite Name       Flux
6               M_C00007__extr                       Oxygen  28.770677
5               M_C00001__extr                          H2O  25.945019
3               M_C00087__extr                       Sulfur  21.199604
0               M_C00011__extr                          CO2   0.554925
7               M_C00288__extr                        HCO3-   0.192824
2               M_C00014__extr                      Ammonia   0.152215
1               M_C00283__extr             Hydrogen sulfide   0.067106
8               M_C00013__extr                  Diphosphate   0.001676
4               M_C14818__extr                         Fe2+   0.000006
2               M_C00059__extr                      Sulfate  21.196760
10              M_C00080__extr                           H+   8.977140
7               M_C00669__extr  gamma-L-Glutamyl-L-cysteine   0.069756
3               M_C00249__extr            Hexadecanoic acid   0.007611
8     

### Check community external metabolites and their reactions 


In [ ]:

for exchange_reaction in community_model.get_exchange_reactions():
    exchange_reaction_object = community_model.reactions.get(exchange_reaction)
    reaction_metabolites = exchange_reaction_object.stoichiometry.keys()
    for metabolite in reaction_metabolites:
        if metabolite:
            reactions = community_model.get_metabolite_reactions(metabolite)
            for reaction in reactions:
                print(community_model.reactions.get(reaction))

### Check community and organisms' biomass reactions


In [ ]:

biomass_reaction = community_model.reactions.get("community_growth")
biomass_metabolite = community_model.metabolites.get("community_biomass")

print(biomass_reaction.)

biomass_reactions = community_model.get_metabolite_reactions("community_biomass")
print("####################### Community_biomass #########################")
for reaction in biomass_reactions:
    print(community_model.reactions.get(reaction))

#print(community_model.get_exchange_reactions())

acaldus_biomass_reactions = community_model.get_metabolite_reactions("M_e_Biomass__extr")
aferrooxidans_biomass_reaction = community_model.get_metabolite_reactions("M_e_Biomass__cytop_aferrooxidans")
print("####################### BIOMASS_cytop_org reactions #########################")
for reaction in acaldus_biomass_reactions:
    print(community_model.reactions.get(reaction))

for reaction in aferrooxidans_biomass_reaction:
    print(community_model.reactions.get(reaction))

### Check community's compartments

In [ ]:
community_model.compartments